In [3]:
import fonction as f
import numpy as np
import os 
import sys


In [4]:
masse_volumique = [3.61 * 10**(-3), 50 * 10**(-3), 80 * 10 **(-3)] #pour 2 classes
E = # pour 2 classes
L = 
r =


rho_l = masse_volumique /  (np.pi * r**2)

I = #moment d'inertie de la corde


In [ ]:
def creer_son(mettre_Tous_les_params):

    Fe = 40000
    MS,CS,KS, phiS_Nx_NmS,NmS,xS = f.corde(mettre_param_corde)
    MB_mod,CB_mod,KB_mod,phiB_NxNy_NmB_mod,NmB_mod,x,y= f.plaque_model(mettre_param_corde)

    t,FextS_NxS_Nt = f.Simu_config(xS, Fe, T=3)

    M_mod,K_mod,C_mod = f.créer_matrice(MS,MB_mod,KS,KB_mod,CS,CB_mod)

    W_mod_a2,Z_mod_a2 = f.UK_params(M_mod,NmS, NmB_mod, phiS_Nx_NmS, phiB_NxNy_NmB_mod, article = False, model = True, mode = 'A2',x = x, y = y)

    Q, F = f.lounch_simu_article(t,FextS_NxS_Nt,phiS_Nx_NmS,NmS,NmB_mod,M_mod,C_mod,K_mod,Z_mod_a2,W_mod_a2)

    FS = F[:NmS,:]
    FB = F[NmS:,:]
    FS_NxS_Nt = phiS_Nx_NmS @ FS
    FB_NxNy_Nt = phiB_NxNy_NmB_mod @ FB
    #display(Audio(FS_NxS_Nt[-1,:], rate=Fe))
    ### save FS_NxS_Nt[-1,:]


